# Generate Training Data

We wish to speed up the process of having a human being name entities that they recognize in a paragraph, and having their positions in the paragraph identified and placed in a syntax usable by spaCy's NER training routine.

## Open PDF file, extract page two and display as sentences

In [26]:
import spacy
import PyPDF2
nlp = spacy.load('en_core_web_sm')

#Open PDF file for reading
pdfFile = open("BarCvDescLJ11.pdf", mode="rb")
pdfReader = PyPDF2.PdfFileReader(pdfFile)

# Select a page to work on
pageNumber = 36

# Get text
OnePage = pdfReader.getPage(pageNumber-1) #0-based count
OnePageText = OnePage.extractText()

# Close PDF file
pdfFile.close()

# Remove newlinesxmx82k-&a. It appears multiple newlines together makes
# Spacy think that is the end of a sentence. The PDF reader reads the text in
# an odd fashion
OnePageText = OnePageText.replace('\n','')

# create a spaCy doc object from the page and break it into sentences
doc = nlp(OnePageText)
l=0
for sent in doc.sents:
     print(l, ": ", sent)
     l = l+1


0 :  36  evaluation it was resistant to scald, net blotch, and powdery mildew, moderately susceptible to BYD, and susceptible to loose smut and leaf rust.
1 :  It was evaluated as Entry 577 in the UC Regional Cereal Testing program from 1981-1984 for  late fall planting in the Central Valley and the south-central coastal regions of California.    
2 :  WESTBRED BARCOTT  
3 :  Westbred Barcott is a six-rowed spring feed barley.
4 :  It was released by Western Plant Breeders.
5 :  It originated as an F2 selection from the male-sterile facilitated recurrent selection population CCXXII-78.
6 :  It was developed to be used in rotation with cotton in the desert southwest.
7 :  It is very early maturing (10 days earlier than Westbred 501).
8 :  It is short with excellent straw strength.
9 :  It has short awns.
10 :  At the time of evaluation it was moderately susceptible to leaf rust, net blotch, powdery mildew, and bacterial leaf blight, and susceptible to stem rust, scald, and BYD.
11 :  It

## Read in Per-line named entity file and match entities to sentence positions.

In [28]:
import re
import csv
import pandas as pd
fname = "Data/barley_p36_ner.txt"

# Covert the nlp senetence generator into a list of sentences
sentences = list(doc.sents)

# Open the file of manually matched pairs (sentence # <tab> word phrase <tab> named entity)
# e.g.:
#  0      AC Metcalfe     CVAR
#  0      two-rowed       TRAT
#  0      barley          CROP
#  1      Agri-Food Candada   ORG
#  1      1997    DATE
file = open(fname)
reader = csv.reader(file, delimiter='\t', quoting=csv.QUOTE_NONE)
data = list()

for row in reader:
    try:
        (sentIndex, phrase, label) = row
        sent = sentences[int(sentIndex)].string.rstrip()
        
        # find all instances of the 'phrase' in the 'sent'.
        iter = re.finditer(r"\b"+phrase+r"\b", sent)
        indices = [m.start(0) for m in iter]
        
        # check to make sure the phrase the user said was there was indeed found
        if len(indices) == 0:
            raise ValueError
                
        # print out all instances
        for i in indices:
#            print(sentIndex, sent, phrase, "("+str(i), i+len(phrase), "'"+label+"')")
            data.append([sentIndex, sent, phrase, "("+str(i)+", "+str(i+len(phrase))+", '"+label+"')"])
            
    except:
        print("Handle manually: ", row)
        
df = pd.DataFrame(data, columns = ["Index", "Sentence", "Phrase", "MatchInfo"])
print(df)


    Index                                           Sentence  \
0       0  36  evaluation it was resistant to scald, net ...   
1       0  36  evaluation it was resistant to scald, net ...   
2       0  36  evaluation it was resistant to scald, net ...   
3       0  36  evaluation it was resistant to scald, net ...   
4       0  36  evaluation it was resistant to scald, net ...   
..    ...                                                ...   
135    63                      The rachilla is short-haired.   
136    64      Hulls are slightly wrinkled to semi-wrinkled.   
137    65                                  Aleurone is blue.   
138    66      Veins on kernels are moderately prominent and   
139    66      Veins on kernels are moderately prominent and   

                     Phrase         MatchInfo  
0                 resistant  (22, 31, 'PPTD')  
1                     scald  (35, 40, 'PATH')  
2                net blotch  (42, 52, 'PATH')  
3            powdery mildew  (58, 72, '

## Create a function to clean up overlapping intervals

In [29]:
import re
coordRegex = re.compile(r'(\d+), (\d+)')

def sortByStart(coords):
    """For use in sort routines, return object with lowest (X,Y) values"""
    # split out coordinates that come in as (5, 7, 'CVAR')
    mo = coordRegex.search(coords)
    return(int(mo.group(1)))

def overlaps(coord1, coord2):
    """Check if coordinates of the form 5, 7, 'CVAR' and 32, 46, 'TRAT' overlap"""
    mo1 = coordRegex.search(coord1)
    mo2 = coordRegex.search(coord2)
    coord1Low = int(mo1.group(1))
    coord1High = int(mo1.group(2))
    coord2Low = int(mo2.group(1))
    coord2High = int(mo2.group(2))
    
    if ((coord1High >= coord2Low) and (coord1Low <= coord2Low) or
        (coord2High >= coord1Low) and (coord2Low <= coord1Low)):
        return True
    else:
        return False

def keepFirst(coord1, coord2):
    """Given overlapping coordinates, return the wider encompassing one."""
    mo1 = coordRegex.search(coord1)
    mo2 = coordRegex.search(coord2)
    coord1Low = int(mo1.group(1))
    coord1High = int(mo1.group(2))
    coord2Low = int(mo2.group(1))
    coord2High = int(mo2.group(2))
 
    if (int(coord1High) - int(coord1Low)) >= (int(coord2High) - int(coord2Low)):
        return True
    else:
        return False

# print("Should be false:", overlaps("(5, 7, 'CVAR')", "(32, 46, 'TRAT')"))
# print("Should be true:", overlaps("(26, 46, 'TRAT')", "(32, 46, 'TRAT')"))
# print("Should be true:", overlaps("(26, 46, 'TRAT')", "(26, 46, 'TRAT')"))
# print("Keeper:", keepFirst("34, 46, 'TRAT'", "34, 46, 'TRAT'"))

def cleanIntervals(inputString=""):
    """order intervals like (5, 7, 'CVAR'), (32, 46, 'TRAT'), (26, 46, 'TRAT') and remove overlapping ones."""
    inputString = inputString.lstrip("(").rstrip(")")
    intervalList = inputString.split("), (")
    intervalList.sort(key = sortByStart)
#    print("Sorted Interval List:", intervalList)

    # Pairwise compare every interval in the list to every other interval to check overlap
    keeperList = [True]*len(intervalList) # Logic array to determine if each interval should be kept
    i=0
    for interval1 in intervalList:
        for interval2 in intervalList:
            if interval1 == interval2:
                if intervalList.index(interval1) != i: # when both are the same we reject the higher one
                    keeperList[i] = False
            else:
                if overlaps(interval1, interval2) and keepFirst(interval1, interval2) == False:
                    keeperList[i] = False
        i = i+1
        
#    print("keeperList:", keeperList)
   
    # Build up the return interval list
    returnStr = "("
    for interval, isKeeper in zip(intervalList, keeperList):
        if isKeeper:
            returnStr = returnStr + interval + "), ("
    return (returnStr.rstrip("), (") + ")")
        
# cleanIntervals("(5, 7, 'CVAR'), (32, 46, 'TRAT'), (5, 9, 'CVAR'), (48, 55, 'ORG'), (26, 46, 'TRAT')")
# cleanIntervals("(0, 8, 'CVAR'), (0, 5, 'CVAR'), (21, 26, 'PLAN'), (32, 37, 'CVAR')")
cleanIntervals("(0, 12, 'CVAR'), (39, 49, 'CVAR'), (39, 49, 'CVAR'), (71, 77, 'CVAR'), (71, 77, 'CVAR'), (92, 113, 'TRAT'), (140, 150, 'CVAR'), (140, 150, 'CVAR'), (181, 187, 'CVAR'), (181, 187, 'CVAR')")


"(0, 12, 'CVAR'), (39, 49, 'CVAR'), (71, 77, 'CVAR'), (92, 113, 'TRAT'), (140, 150, 'CVAR'), (181, 187, 'CVAR')"

## Aggregate all matches for each sentence on a single line and output in spaCy training format

In [30]:
# use Pandas dataframes to aggregate all entity matches together for a single sentence
agg_rules = {'Sentence': 'first', 'Phrase': 'first', 'MatchInfo': lambda x: ', '.join(x)}
res = df.groupby('Index').agg(agg_rules)
#print(res)

# Now format it just like what is needed for the spaCy training module: 
# E.g.:
# ('Eight-Twelve is a six-rowed winter feed barley', {'entities': [(0, 12, 'CVAR'), (18, 27, 'TRAT'), (28, 39, 'TRAT'),(40, 46, 'CROP')]}),
records = res.to_dict('records')
for i in range(0,len(records)):
    print("    ('"+records[i]['Sentence']+"', {'entities': ["+cleanIntervals(records[i]['MatchInfo'])+"]}),")
    #print("    ('"+records[i]['Sentence']+"', {'entities': ["+records[i]['MatchInfo']+"]}),")


    ('36  evaluation it was resistant to scald, net blotch, and powdery mildew, moderately susceptible to BYD, and susceptible to loose smut and leaf rust.', {'entities': [(22, 31, 'PPTD'), (35, 40, 'PATH'), (42, 52, 'PATH'), (58, 72, 'PATH'), (74, 96, 'PPTD'), (100, 103, 'PATH'), (109, 120, 'PPTD'), (124, 134, 'PATH'), (139, 148, 'PATH')]}),
    ('At the time of evaluation it was moderately susceptible to leaf rust, net blotch, powdery mildew, and bacterial leaf blight, and susceptible to stem rust, scald, and BYD.', {'entities': [(33, 55, 'PPTD'), (59, 68, 'PATH'), (70, 80, 'PATH'), (82, 96, 'PATH'), (102, 123, 'PATH'), (129, 140, 'PPTD'), (144, 153, 'PATH'), (155, 160, 'PATH'), (166, 169, 'PATH')]}),
    ('Westbred Gustoe is a six-rowed spring feed barley.', {'entities': [(0, 15, 'CVAR'), (21, 30, 'TRAT'), (31, 37, 'TRAT'), (38, 42, 'TRAT'), (43, 49, 'TRAT')]}),
    ('It was released by Western Plant Breeders in 1982.', {'entities': [(19, 41, 'ORG'), (45, 49, 'DATE')]}),
    ('Its e